In [9]:
import sys
sys.path.append("./")
sys.path.append("./hw2/stud/")
from torch.utils.data import DataLoader
import config
# from tqdm.auto import tqdm
import numpy as np
from utils import *
from load import *
# from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning import Trainer
from glossBERT import GlossBERT


In [10]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# FINE_TRAIN_DATA
# FINE_VAL_DATA
# FINE_TEST_DATA


In [11]:
print(config.DEVICE)

cuda


In [12]:
seed_everything(config.SEED)

In [13]:

train_dataset = FineGrainedDataset(config.FINE_TRAIN_DATA)
val_dataset = FineGrainedDataset(config.FINE_VAL_DATA)
test_dataset = FineGrainedDataset(config.FINE_TEST_DATA)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=glossBERT_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=glossBERT_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=glossBERT_collate_fn)

mapping = load_map(config.MAP_PATH)


KeyboardInterrupt: 

In [ ]:
# take a look at the data 
#load one data fr om the train_dataset
# print(train_dataset[0])

In [ ]:
# # # display one item
# for batch in train_loader:
#     input_ids, senses_ids, labels, attention_mask = batch
#     # pritn one item
#     print(input_ids)
#     print(senses_ids)
#     print(labels)
#     print(attention_mask)

#     break



In [ ]:
model = GlossBERT()
# model.to(config.DEVICE)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [15]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# TRAINER
# wandb_logger = WandbLogger(project='nlp_hw2', log_model=True)
trainer = Trainer(gpus=0, max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])#, logger=wandb_logger)
trainer.fit(model, train_loader, val_loader)
# trainer = Trainer(max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])
# trainer.fit(model, train_loader, val_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1566: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name       | Type                             | Params
----------------------------------------------------------------
0 | roberta    | RobertaForSequenceClassification | 124 M 
1 | bert       | RobertaModel                     | 124 M 
2 | relu       | ReLU                             | 0     
3 | classifier | Linear                           | 1.5 K 
4 | loss       | CrossEntropyLoss                 | 0     
----------------------------------------------------------------
249 M     Trainable params
0         Non-trainable params
249 M     Total params
997.180   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 4/5909 [00:08<3:37:12,  2.21s/it, loss=0.655, v_num=19]

In [13]:
from transformers import AutoTokenizer
import config 
sentence = "The bank can guarantee deposits will eventually cover future tuition costs because it invests in adjustable-rate mortgage securities."
sense = "bank%1:14:00::"
definition = "a financial institution that accepts deposits and channels the money into lending activities"
max_seq_length = 128
target_idx = 1

tokenizer = AutoTokenizer.from_pretrained("roberta-base",  do_lower_case=True)

tokens_a = tokenizer.tokenize(sentence)
tokens_b = tokenizer.tokenize(definition)

tokens = [config.CLS_TOKEN] + tokens_a + ["[SEP]"]
segment_ids = [0] * len(tokens)
tokens += tokens_b + ["[SEP]"]
segment_ids += [1] * (len(tokens_b) + 1)

input_ids = tokenizer.convert_tokens_to_ids(tokens)

# The mask has 1 for real tokens and 0 for padding tokens. Only real
# tokens are attended to.
input_mask = [1] * len(input_ids)

padding = [0] * (max_seq_length - len(input_ids))
input_ids += padding
input_mask += padding
segment_ids += padding



In [11]:
import torch
from transformers import RobertaForSequenceClassification, AutoModel

model = AutoModel.from_pretrained("roberta-base", type_vocab_size=1, ignore_mismatched_sizes=True)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
max_seq_length = 128

tokenizer = AutoTokenizer.from_pretrained("roberta-base",  do_lower_case=True)

tokens_a = tokenizer.tokenize(sentence)
tokens_b = tokenizer.tokenize(definition)

tokens = [config.CLS_TOKEN] + tokens_a + ["[SEP]"]
segment_ids = [0] * len(tokens)
tokens += tokens_b + ["[SEP]"]
segment_ids += [1] * (len(tokens_b) + 1)

# truncate input_ids, input_mask, and segment_ids to max_seq_length
if len(tokens) > max_seq_length:
    tokens = tokens[:max_seq_length]
    segment_ids = segment_ids[:max_seq_length]

input_ids = tokenizer.convert_tokens_to_ids(tokens)

# The mask has 1 for real tokens and 0 for padding tokens. Only real
# tokens are attended to.
input_mask = [1] * len(input_ids)

# pad input_ids, input_mask, and segment_ids to max_seq_length
padding = [0] * (max_seq_length - len(input_ids))
input_ids += padding
input_mask += padding
segment_ids += padding

out = model(torch.tensor([input_ids]), torch.tensor([input_mask]), torch.tensor([segment_ids]))


IndexError: index out of range in self

In [18]:
from transformers import BertConfig, BertModel

config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2, type_vocab_size=2)

#print config
bert = BertModel.from_pretrained("bert-base-uncased", config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
